# The Structure of fMRI Data

Before we get started discussing the pre-processing of fMRI data, it is useful to have a review of the structure of an fMRI image. In this lesson we are going to talk a lot about the spatial and temporal elements of fMRI datasets, with lots of references to the fMRI timeseries. So it is important you understand what is meant by this before we go any further.

## 4D images

At its most basic, an fMRI dataset is a collecting of 3D volumes of the brain that were collected in rapid succession. The speed at which we collect these volumes is given by the TR of sequence, and is usually around 2-3 seconds. This gives the temporal resolution of the data. Together, these 3D images form a 4D dataset, where we have three spatial dimensions and a fourth dimension of time. As we move through these 3D volumes we see how the BOLD signal changed over the course of the experiment. Importantly, each of these 3D volumes is collected while the subject is engaged with some sort of cognitive task in the scanner. What these tasks look like and how we use them is the focus of the Experimental Design and Optimisation unit. For the purposes of this lesson, just know that the subject was doing something while in the scanner. Our broad purpose in analysing the data is to try and match changes in the BOLD signal over time with changes in the different conditions of this experimental task.
The fMRI timeseries

Each of the 3D volumes that make up an fMRI dataset can be thought of as a snapshot in time, indicating the values of the BOLD signal at that precise moment. A typical fMRI scan may amass around 200 of these snapshots. For each voxel in the image, we can see how the signal at that point in the image changed over time by looking at how its value shifts across all the 3D volumes. For a scan with 200 volumes, we therefore have 200 values for each voxel in the image documenting how the BOLD signal changed over the course of the experiment. This sequence of values is known as the fMRI timeseries. The timeseries at each voxel represents the raw data that we will subject to statistical analysis to try and discern patterns that map on to the pattern of the experimental conditions. For now, just consider the scale of this data. A typical fMRI volume may have dimensions $60 \times 60 \times 40$ with 200 datapoints at each voxel. This gives over 28 million data points in a single fMRI dataset. An example of the timeseries for 3 voxels is given below.